In [19]:
import torch
import torchvision
from torch import nn

In [10]:
train_data=torchvision.datasets.CIFAR10(root='../data',train=True,transform=torchvision.transforms.ToTensor(),download=True)
test_data=torchvision.datasets.CIFAR10(root='../data',train=False,transform=torchvision.transforms.ToTensor(),download=True)

In [17]:
from torch.utils.data import DataLoader

train_dataloader=DataLoader(train_data,batch_size=64,shuffle=True)
test_dataloader=DataLoader(test_data,batch_size=64,shuffle=True)

6

In [20]:

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.sequential = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=32,kernel_size=5,stride=1,padding=2),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=5,stride=1,padding=2),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5,stride=1,padding=2),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Flatten(start_dim=1),
            nn.Linear(in_features=64*4*4,out_features=10)
        )
    def forward(self,x):
        out=self.sequential(x)
        return out
    def